<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/S105ocrCorrectionV03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# correction of ocr-generated text 
- using correction dictionaries 
- using rules


In [ ]:
import re, os, sys


In [ ]:
!wget https://heibox.uni-heidelberg.de/f/d601ceb0af5a4671a8e7/?dl=1
!mv index.html?dl=1 Parfum_Arm_ABBY.txt

!wget https://heibox.uni-heidelberg.de/f/743a1a57a37c42d8b585/?dl=1
!mv index.html?dl=1 Parfum_Armenian_uncorrected.txt



In [ ]:
# dealing with Armenian OCR output with line breaks (is it correct?)

FName = 'Parfum_Armenian_uncorrected.txt'
FNameOut = 'Parfum_Armenian.txt'

FIn = open(FName, 'r')
FOut = open(FNameOut, 'w')

for SLine in FIn:
    SLine = SLine.strip()
    if SLine == '': 
        FOut.write('\n\n')
        continue
    if SLine[-1] == '-':
        SLine2write = SLine[:-1]
        FOut.write(SLine2write)
        continue

    FOut.write(SLine + ' ')
FOut.flush()



In [ ]:
!wc Parfum_Arm_ABBY.txt
!wc Parfum_Armenian_uncorrected.txt
!wc Parfum_Armenian.txt

  1489  68852 867013 Parfum_Arm_ABBY.txt
 13592  72207 854251 Parfum_Armenian_uncorrected.txt
  6126  69006 849250 Parfum_Armenian.txt


In [ ]:
!wget https://heibox.uni-heidelberg.de/f/c977e87cf2b244e6801b/?dl=1
!mv index.html?dl=1 KorpusARM.tgz


In [ ]:
!tar xvzf KorpusARM.tgz
!mkdir KorpusARM1
!mkdir KorpusARM1/stage01
# concatenating files
!cat korpusARM/hyFiktion/* >KorpusARM1/stage01/hyFiktion.txt
!cat korpusARM/hyNatur/* >KorpusARM1/stage01/hyNatur.txt
!cat korpusARM/hyRecht/* >KorpusARM1/stage01/hyRecht.txt
!mkdir KorpusARM1/stage02

# function for Armenian line breaks:

def correctLineBreaksHY(FName, FNameOut):
    FIn = open(FName, 'r')
    FOut = open(FNameOut, 'w')
    countHyphens = 0
    for SLine in FIn:
        SLine = SLine.strip()
        if SLine == '': 
            FOut.write('\n\n')
            continue
        if SLine[-1] == '-':
            SLine2write = SLine[:-1]
            FOut.write(SLine2write)
            countHyphens +=1
            continue
        FOut.write(SLine + ' ')
    FOut.flush()
    print(str(countHyphens) + ' hyphens corrected')
    return

correctLineBreaksHY('KorpusARM1/stage01/hyFiktion.txt', 'KorpusARM1/stage02/hyFiktion.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyNatur.txt', 'KorpusARM1/stage02/hyNatur.txt')
correctLineBreaksHY('KorpusARM1/stage01/hyRecht.txt', 'KorpusARM1/stage02/hyRecht.txt')


In [ ]:

!wc KorpusARM1/stage02/hyFiktion.txt
!wc KorpusARM1/stage02/hyNatur.txt
!wc KorpusARM1/stage02/hyRecht.txt

   6208   92131 1081755 KorpusARM1/stage02/hyFiktion.txt
  3642  67142 870081 KorpusARM1/stage02/hyNatur.txt
   8940   86621 1288655 KorpusARM1/stage02/hyRecht.txt


In [ ]:
# Dowloading the file with corrections
!wget https://heibox.uni-heidelberg.de/f/14706c04a4024b2f937d/?dl=1
!mv index.html?dl=1 Pilot-Corrections-all.tsv
!wc Pilot-Corrections-all.tsv

In [ ]:
def readCorrections(colNumberOri, colNumberCorrect, SFIn, SFOut = None):
    LTWrongCorrect = []
    DWrongCorrect = {}
    FOut = open(SFOut, 'w')
    with open(SFIn, 'r') as FIn:
        count = 0
        for SLine in FIn:
            count += 1
            if count == 1: continue
            SLine = SLine.strip()
            LLine = SLine.split('\t')
            SWrong = LLine[colNumberOri]
            SCorrect = LLine[colNumberCorrect]
            if SWrong != '' and SCorrect != '' and SWrong != SCorrect:
                TWrongCorrect = (f'[{SWrong}]', f'[{SCorrect}]')
                LTWrongCorrect.append(TWrongCorrect)
                if SWrong in DWrongCorrect.keys():
                    SCorrect1 = DWrongCorrect[SWrong]
                    if SCorrect1 != SCorrect:
                        print(SWrong + '\t' + SCorrect1 + '\t' + SCorrect)
                DWrongCorrect[SWrong] = SCorrect
    if SFOut:
        for SWrong, SCorrect in LTWrongCorrect:
            FOut.write(f'{SWrong}\t{SCorrect}\n')    
        FOut.flush()
    print(len(DWrongCorrect))

    return LTWrongCorrect, DWrongCorrect

def readCorrectionsFrq(colNumberOri, colNumberCorrect, colNumberFrq, SFIn, SFOut = None):
    LTWrongCorrect = []
    '''
    if type(LTWrongCorrect) == list:
        pass
    '''

    DWrongCorrect = {}
    FOut = open(SFOut, 'w')
    with open(SFIn, 'r') as FIn:
        count = 0
        for SLine in FIn:
            count += 1
            if count == 1: continue
            SLine = SLine.rstrip('\n')
            LLine = SLine.split('\t')
            SWrong = LLine[colNumberOri]
            SCorrect = LLine[colNumberCorrect]
            SFrq = LLine[colNumberFrq]
            if SWrong != '' and SCorrect != '' and SWrong != SCorrect:
                TWrongCorrect = (f'[{SWrong}]', f'[{SCorrect}]', f'{SFrq}')
                LTWrongCorrect.append(TWrongCorrect)
                if SWrong in DWrongCorrect.keys():
                    SCorrect1 = DWrongCorrect[SWrong]
                    if SCorrect1 != SCorrect:
                        print(SWrong + '\t' + SCorrect1 + '\t' + SCorrect)
                DWrongCorrect[SWrong] = SCorrect
    if SFOut:
        for SWrong, SCorrect, SFrq in LTWrongCorrect:
            FOut.write(f'{SWrong}\t{SCorrect}\t{SFrq}\n')    
        FOut.flush()
    print(len(DWrongCorrect))

    return LTWrongCorrect, DWrongCorrect


In [ ]:
LTWrongCorrectWordF, DWrongCorrectWordF = readCorrectionsFrq(1, 4, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-WordForm.tsv')
# LTWrongCorrectLemmaF, DWrongCorrectLemmaF = readCorrectionsFrq(3, 6, 9, '/content/Pilot-Corrections-all.tsv', SFOut = 'Pilot-Corrections-all-Lemma.tsv')
print(LTWrongCorrectWordF)
# print(LTWrongCorrectLemmaF)
# ինձ|ինչ
# առջև|առջևից
# ինչ|ինձ
# ինչ|ես
# գիտենալ|իմանալ


In [ ]:
print(len(DWrongCorrectWordF))
# print(len(DWrongCorrectLemmaF))


In [ ]:
for key, value in sorted(DWrongCorrectWordF.items()):
    print(f'{key}\t{value}')

In [ ]:
'''
for i in range(5):
    LKeys = list(DWrongCorrectLemmaF.keys())
    SKey = LKeys[i]
    SVal = DWrongCorrectWordF[SKey]
    print(f'{SKey} {SVal}\n')
'''
 

In [ ]:

def tokenizeTextHY(SFIn):
    LLParagraphs = []
    with open(SFIn, 'r') as FIn:
        for SLine in FIn:
            SLine = SLine.strip()
            if SLine == '': continue
            LLine = re.split('([ ,\.\:։;\'\"\(\)\-\–\!\?\{\}\t\«\»]+)', SLine)
            # if LLine == '': continue
            if LLine: LLParagraphs.append(LLine)
    return LLParagraphs

def printLLParagraphs(SFIn, SFOut, DCorrections1 = None, DCorrections2 = None):
    LLParagraphs = tokenizeTextHY(SFIn)
    LCorrected = []
    FOut = open(SFOut, 'w')
    for LPara in LLParagraphs:
        # if LPara == [] or LPara == ['']: continue
        if LPara == []: continue
        # FOut.write(str(LPara) + '\n')
        LParaNew = []
        FOut.write('<p>\n')
        
        i = 0 # counting words
        for el in LPara:
            i+=1 # index of next word
            # print(el, i)
            # if el == ' ': continue
            if DCorrections1:
                if el in DCorrections1:
                    iStart = i-5
                    if iStart <0: iStart = 0
                    iFinish = i+5
                    # if iFinish >len(LPara): iFinish = len(LPara)
                    LContext = LPara[iStart : iFinish]
                    elCorrect = DCorrections1[el]
                    LCorrected.append((el, elCorrect, LContext, LPara))

                else:
                    elCorrect = el
            '''
            if DCorrections2:
                if el in DCorrections2:
                    elCorrect = DCorrections2[el]
                    LCorrected.append((el, elCorrect, LPara))
                else:
                    elCorrect = el                    
            LParaNew.append(elCorrect)

            '''


        SParaNew = ' '.join(LParaNew)
        FOut.write(f'{SParaNew}\n')

        FOut.write('</p>\n')
    FOut.flush()
    return LCorrected

In [ ]:
def printCorrections(LCorrected, SFOut):
    FOut1 = open(SFOut, 'w')
    for (el, elCorr, LContext, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        SContext = ''.join(LContext)
        FOut1.write( f'{el} \t {elCorr} \n\tcontext: {SContext} \n\tparagraph: {SPara}\n\n' )    
    return

In [ ]:
LCorrected1 = printLLParagraphs('/content/Parfum_Arm_ABBY.txt', '/content/Parfum_Arm_ABBY-corrected.txt', DWrongCorrectWordF)
printCorrections(LCorrected1, '/content/Parfum_Arm_ABBY-changes.txt')
print(len(LCorrected1))
LCorrected2 = printLLParagraphs('/content/Parfum_Armenian.txt', '/content/Parfum_Armenian-corrected.txt', DWrongCorrectWordF)
printCorrections(LCorrected2, '/content/Parfum_Armenian-changes.txt')
print(len(LCorrected2))


6
918


In [ ]:

'''
print(len(LCorrected1))
if LCorrected1:
    FOut1 = open('/content/Parfum_Arm_ABBY-changes.txt', 'w')
    for (el, elCorr, LContext, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        SContext = ''.join(LContext)
        FOut1.write( f'{el}\t{elCorr}\t{SContext}\t{SPara}\n\n' )

print(len(LCorrected2))
if LCorrected2:
    FOut2 = open('/content/Parfum_Armenian-changes.txt', 'w')
    for (el, elCorr, LContext, LPara) in LCorrected2:
        SPara = ''.join(LPara)
        SContext = ''.join(LContext)        
        FOut2.write( f'{el}\t{elCorr}\t{SContext}\t{SPara}\n\n' )
'''

'''
if LCorrected1:
    FOut1 = open('/content/KorpusARM1/stage02/hyFiktion-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )
'''

'''
if LCorrected1:
    FOut1 = open('/content/KorpusARM1/stage02/hyNatur-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )
'''


'''
if LCorrected1:
    FOut1 = open('/content/KorpusARM1/stage02/hyRecht-changes.txt', 'w')
    for (el, elCorr, LPara) in LCorrected1:
        SPara = ''.join(LPara)
        FOut1.write( f'{el}  {elCorr}\t\t{SPara}\n\n' )
'''


In [ ]:
# LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyFiktion.txt', '/content/KorpusARM1/stage02/hyFiktion-corrected.txt', DWrongCorrectWordF, DWrongCorrectLemmaF)
LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyFiktion.txt', '/content/KorpusARM1/stage02/hyFiktion-corrected.txt', DWrongCorrectWordF)
print(len(LCorrected1))
printCorrections(LCorrected1, '/content/KorpusARM1/stage02/hyFiktion-changes.txt')



1138


In [ ]:
# LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyNatur.txt', '/content/KorpusARM1/stage02/hyNatur-corrected.txt', DWrongCorrectWordF, DWrongCorrectLemmaF)
LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyNatur.txt', '/content/KorpusARM1/stage02/hyNatur-corrected.txt', DWrongCorrectWordF)
print(len(LCorrected1))
printCorrections(LCorrected1, '/content/KorpusARM1/stage02/hyNatur-changes.txt')



616


In [ ]:
# LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyRecht.txt', '/content/KorpusARM1/stage02/hyRecht-corrected.txt', DWrongCorrectWordF, DWrongCorrectLemmaF)
LCorrected1 = printLLParagraphs('/content/KorpusARM1/stage02/hyRecht.txt', '/content/KorpusARM1/stage02/hyRecht-corrected.txt', DWrongCorrectWordF)
print(len(LCorrected1))
printCorrections(LCorrected1, '/content/KorpusARM1/stage02/hyRecht-changes.txt')



733
